## Agents and Tools

### Agents: Agents enable LLMs to work with tools that perform various tasks
To use agents, we require three things:
- A base LLM,
- A tool that we will be interacting with,
- An agent to control the interaction.

### In the following example, we will use the Math tool to perform some simple math operations

In [1]:
from dotenv import load_dotenv
from langchain_openai import OpenAI
load_dotenv()

True

In [32]:
'''llm = OpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo-instruct"
)'''
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

In [33]:
from langchain.chains import LLMMathChain # Chain that interprets a prompt and executes python code to do math.
from langchain.agents import Tool

In [34]:
# initialize the math tool

llm_math = LLMMathChain.from_llm(llm=llm)

math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)

# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]  # is a list of possible Tools that your agent can use

In [35]:
print(tools[0].name, "---", tools[0].description)

Calculator --- Useful for when you need to answer questions about math.


## Now create an Agent
#### Zero-shot ReAct
The Zero-shot ReAct Agent is a language generation model that can create realistic contexts even without being trained on specific data. It can be used for various tasks such as generating creative text formats, language translation, and generating different types of creative content.

In [42]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="chat-zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10,
    handle_parsing_errors=True
)

In [43]:
zero_shot_agent("What is root over 25?")



> Entering new AgentExecutor chain...
Question: What is root over 25?
Thought: I can use the calculator to find the square root of 25. 
Action:
```json
{
  "action": "Calculator",
  "action_input": "sqrt(25)"
}
```

Observation: Answer: 5.0
Thought:Thought: I now know the final answer.
Final Answer: 5.0 


> Finished chain.


{'input': 'What is root over 25?', 'output': '5.0'}

In [44]:
problem = """
        You are building a house. There are two bedrooms of 5 metres by 5 metres each and drawing cum open kitchen 
        is 7 metres by 6 metres and balcony of 3 metres by 2 metres. 
        What is the total area of your house?
        """
zero_shot_agent(problem)



> Entering new AgentExecutor chain...
Question: You are building a house. There are two bedrooms of 5 metres by 5 metres each and drawing cum open kitchen  is 7 metres by 6 metres and balcony of 3 metres by 2 metres.  What is the total area of your house?
Thought: I have to calculate area of each room first and then add them up.
Thought: Area of each bedroom is 5*5 = 25 sq. meter
Thought: Area of two bedrooms is 25*2 = 50 sq. meter
Thought: Area of drawing cum open kitchen is 7*6 = 42 sq. meter
Thought: Area of balcony is 3*2 = 6 sq. meter
Thought: So, total area of the house is 50+42+6 = 98 sq. meter
Final Answer: 98 sq. meter 


> Finished chain.


{'input': '\n        You are building a house. There are two bedrooms of 5 metres by 5 metres each and drawing cum open kitchen \n        is 7 metres by 6 metres and balcony of 3 metres by 2 metres. \n        What is the total area of your house?\n        ',
 'output': '98 sq. meter'}

## Using mutiple Tools
In the following example, we will be using the Duck Duck Go search API as a Tool combined with the Math Tool
We will then give the Agent a problem that initiates a search first followed by a Calculation.

In [45]:
# Define the Tools

from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search_tool = Tool.from_function(
    func=search.run,
    name="Search",
    description="useful for when you need to search the internet for information"
)

llm_math_chain = LLMMathChain(llm=llm, verbose=True)

math_tool = Tool.from_function(
    func=llm_math_chain.run,
    name="Calculator",
    description="Useful for when you are asked to perform math calculations"
)

/Users/Manas/.local/share/virtualenvs/Self-Learning-rjN_b9yD/lib/python3.9/site-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [47]:
# Define the Agent

tools = [search_tool, math_tool]

agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=True
)

In [49]:
# Run the Agent with the problem

agent.run("""Get Microsoft Stock Price taken from Google Finance and display in both USD and INR values""")



> Entering new AgentExecutor chain...
Thought: I need to search for Microsoft's stock price on Google Finance and convert it to INR. 
Action: Search
Action Input: {'query': 'Microsoft stock price Google Finance'} 
Observation: Step 2 - Publishing the Google Finance Stock Prices Data. Go to File, select Share and choose Publish to web. A dialog box will appear. Click on the Publish button. Press Ctrl + C to copy the link. Step 3 - Importing Stock Prices into Excel. Open an Excel file, go to the Data tab, and select From Web. A dialog box will appear. The GOOGLEFINANCE function is an amazing tool used to make Google Sheets stock formulas for people who need to work with real-time finance data. This single function gives you not only the current prices of stocks and conversion rates of currencies, but also historical data of these securities over days, weeks, months or years. For example, you want to import the stock price or value of Facebook Inc. traded on NASDAQ. The ticker symbol fo

'The price of Microsoft stock is approximately USD 432.68 and INR 36140.66.'

## Create a custom tool

In [50]:
from langchain.tools import BaseTool
from langchain_core.output_parsers import JsonOutputParser
from langchain.tools import tool

@tool("JSON_Response", return_direct=True)
def StructuredResponseTool(question: str):
    """
    use this tool to send a prompt and get a JSON returned 
    with three fields - Topic, Question_Details and Detailed_Response
    """
    json_prompt = PromptTemplate.from_template(
    """Return a JSON object with an `answer` key that answers the following question: {question}. 
        The JSON object will have three fields - Topic, Question_Details and Detailed_Response"""
    )

    model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.5)

    json_parser = JsonOutputParser()
    json_chain = json_prompt | model | json_parser
    x = json_chain.invoke({"question": question})
    return x

In [51]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [52]:
from langchain.agents import initialize_agent

tools = [StructuredResponseTool]

# initialize agent with tools
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10
)

In [53]:
zero_shot_agent.run(input="Which is the top football team in the world and how, the response should have three fields in the JSON - topic of the question, the question and the detailed response")



> Entering new AgentExecutor chain...
Thought: I can answer this question by using the JSON_Response tool to get the information. 

Action: JSON_Response
Action Input: {"question": "Which is the top football team in the world and how?"} 
Observation: {'Topic': 'Football', 'Question_Details': 'Which is the top football team in the world and how?', 'Detailed_Response': "According to the latest FIFA rankings, the top football team in the world is Belgium. This is determined by a points system based on a team's performance in international matches. Belgium has consistently performed well in major tournaments and has a talented squad of players, including Kevin De Bruyne and Eden Hazard. They also have a strong team dynamic and a successful coaching staff, which has helped them maintain their top ranking."}


> Finished chain.


{'Topic': 'Football',
 'Question_Details': 'Which is the top football team in the world and how?',
 'Detailed_Response': "According to the latest FIFA rankings, the top football team in the world is Belgium. This is determined by a points system based on a team's performance in international matches. Belgium has consistently performed well in major tournaments and has a talented squad of players, including Kevin De Bruyne and Eden Hazard. They also have a strong team dynamic and a successful coaching staff, which has helped them maintain their top ranking."}